convert keras h.5 to Tensorflow .pb

In [ ]:
import keras
import tensorflow as tf
print(keras.__version__)
print(tf.__version__)

In [ ]:
model = tf.keras.models.load_model("../Model_CNN_7class.h5", compile=False)    

In [ ]:
model.save("tf_model")

In [ ]:
! dir tf_model

 view Model graph input & Output

In [ ]:
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
# import tensorflow.compat.v1 as tf
from tensorflow.python.framework import graph_util

In [ ]:
path = "D:\Kuliah\Semester 7\Proyek\FaceRcognition\FaceRecognitionCnnModel\Optimized Model\tf_model"
importer = tf.compat.v2.saved_model.load("tf_model/")

In [ ]:
infer = importer.signatures['serving_default']

In [ ]:
f = convert_variables_to_constants_v2(infer)
graph_def = f.graph.as_graph_def()

for i in graph_def.node:
    print(i.name, i.op)

In [ ]:
f = convert_variables_to_constants_v2(infer)
graph_def = f.graph.as_graph_def()

graph_def

convert Model .pb to frozen model graph .pb 

In [ ]:
from tensorflow.python.tools import optimize_for_inference_lib

In [ ]:
f = convert_variables_to_constants_v2(infer)
graph_def = f.graph.as_graph_def()

graph_def = optimize_for_inference_lib.optimize_for_inference(graph_def,
                                                             ['conv2d_input'],
                                                             ['Identity'],
                                                             tf.float32.as_datatype_enum)

with tf.io.gfile.GFile('frozen_graph.pb', 'wb') as f:
    f.write(graph_def.SerializeToString())

inference model using opencv DNN, open cv using new version

In [ ]:
import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
cv.__version__

Load model 

In [ ]:
labels = ["Syahrul Gunawan Ramdhani", 
         "Harsel Octavian", 
         "Latif Ma'ruf",
         "Muhammad Rizqi",
         "Ravel Ariondo",
         "Robi Ardiansyah",
         "Tatas Handaru"]

In [ ]:
facerecognition_model = "frozen_graph.pb"
net = cv.dnn.readNet(facerecognition_model)

In [ ]:
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)

In [ ]:
layerOutput = net.getUnconnectedOutLayersNames()

In [ ]:
layerOutput

predict image using opencv dnn

In [ ]:
def detectFace(img):
#     img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
#     img = cv.resize(img, (50,50))
    target_size = (50, 50)

    resized = cv.resize(img, target_size)
    return resized

In [ ]:
os.listdir("test_data/")

In [ ]:
img = cv.imread("test_data/Harsel.jpg", cv.IMREAD_GRAYSCALE)
img = detectFace(img)
blob = cv.dnn.blobFromImage(img, 1.0, (50,50), (0,0,0), swapRB=True, crop=False)

In [ ]:
img.shape, blob.shape

In [ ]:
net.setInput(blob)
output = net.forward(layerOutput)

In [ ]:
output

In [ ]:
idx = output[0].argmax(axis=1)[0]

confidence = output[0].max(axis=1)[0]*100

In [ ]:
plt.imshow(img, cmap="gray")
plt.title(f"label : {labels[idx], confidence}")

Apply to Video Frame

In [ ]:
def draw_ped(img, label, x0, y0, xt, yt, color=(255, 127, 0), text_color=(255, 255, 255)):
    (w, h), baseline = cv.getTextSize(label, cv.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    cv.rectangle(img,
                 (x0, y0 + baseline),
                 (max(xt, x0 + w), yt),
                 color,
                 2)
    
    cv.rectangle(img, 
                 (x0, y0 -h),
                 (x0 + w, y0 + baseline),
                 color, 
                 -1)
    
    cv.putText(img,
               label,
               (x0, y0),
               cv.FONT_HERSHEY_COMPLEX,
               0.5,
               text_color,
               1,
               cv.LINE_AA)

    return img

In [ ]:
# -----load harcascade Model-----
face_cascade = cv.CascadeClassifier('../haarcascade_frontalface_default.xml')

# ------load tensorflow optimize model-----
facerecognition_model = 'frozen_graph.pb'
net = cv.dnn.readNet(facerecognition_model)
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)
layerOutput = net.getUnconnectedOutLayersNames()

cap = cv.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        gray = cv.cvtColor(frame, cv.COLOR_RGB2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 5)
        for (x, y, w, h) in faces:
            face_img = gray[y:y+h, x:x+w]
            face_img = cv.resize(face_img, (50, 50))

            blob = cv.dnn.blobFromImage(face_img, 1.0, (50, 50), (0, 0, 0), swapRB=True, crop=False)
            net.setInput(blob)
            output = net.forward(layerOutput)
            idx = output[0].argmax(axis=1)[0]
            confidence = output[0].max(axis=1)[0]*100

            if confidence > 80:
                label_text = "%s (%.2f %%)" % (labels[idx], confidence)
            else:
                label_text = "N/A"
            frame = draw_ped(frame, label_text, x, y, x + h, y + h, color=(0, 255, 255), text_color=(50, 50, 50))
        cv.imshow('Face Recognition', frame)
    else:
        break
    if cv.waitKey(10) == ord('q'):
        break
cv.destroyAllWindows()
cv.release()

In [ ]:
import pandas as pd

labels = ["Syahrul Gunawan Ramdhani", 
         "Harsel Octavian", 
         "Latif Maruf",
         "Muhammad Rizqi",
         "Ravel Ariondo",
         "Robi Ardiansyah",
         "Tatas Handaru"]
labels_df = pd.Datafr